# Segmentation

## 개요

* Image classification, object detection은 물체의 종류를 분류하거나 물체의 존재, 위치를 탐지.
* Segmentation : 픽셀 수준에서 이미지의 각 부분이 어떤 의미를 갖는 영역인지 분리하는 방법
* 이미지 분할 기술 -  ex) 사람과 배경을 분리

### 실습 목표

* 세그멘테이션의 방식을 공부합니다.
* 시맨틱 세그멘테이션 모델의 결괏값을 이해합니다.
* 시맨틱 세그멘테이션을 위한 접근 방식을 이해합니다.

### 학습 내용

1. 세그멘테이션 문제의 종류
2. 주요 세그멘테이션 모델
   * FCN
   * U-NET
   * DeepLab 계열
3. 세그멘테이션의 평가
4. Upsampling의 다양한 방법

## Segmentation 문제의 종류

### Semantic Segmentation

* 어떤 물체들이 모여있는 영역의 위치를 인식
* 물체들이 어떤 물체인지 판별, 같은 물체끼리 각각을 구분하지 않음
* U-Net
  - 입력으로 572x572 크기의 이미지
  - 출력으로 388x388 크기의 두가지 클래스를 가진 Segmentation map
  - 클래스에 따른 Semantic Segmentation Map을 얻을 수 있음
  - 큰 출력 값 : Segmentation을 위한 Image의 각 pixel 영역의 Class 정보가 필요

### Instance Segmentation

* 같은 class 내에서도 각 개체(instance)를 분리하여 segmentation 수행.
* Mask-R-CNN
  - 2-stage Object Detection인 Faster R-CNN을 계승
  - Region of Interest Pooling Layer(RoIPool) 개념을 개선
  - 1. 정확한 segmentation에 유리한 RoIAlign
  - 2. Class 별 mask 분리
  - Class 별 Object detection과 semantic segmentation을 사실상 하나의 Task로 엮어낸 것으로 평가받는 중요한 모델

* RoIPool Layer : 다양한 RoI 영역을 Pooling을 통해 동일한 크기의 Feature map으로 추출해 내는 Layer.
* 고정 size의 Feature map을 바탕으로 bounding box와 object class를 추론
* Object 영역의 정확한 masking을 필요로하는 segmentation에서는 문제가 발생
* RoIPool 과정에서 Quantization(양자화)가 필요
* 정해진 grid로 분할된 구역에서 있는 RoI 영역이 무조건 해당 grid의 배수인 것은 아니다
* grid로 분할된 영역 중 절반 이상이 RoI 영역에 덮히는 곳으로 끼워 맞추다보면, 어쩔 수 없이 RoI 영역 밖에 포함되어 있는 경우도 있고 자투리 영역이 버려지기도 한다.
* 이런 상황은 필연적으로 semantic segmentation의 정보손실과 왜곡을 야기

<img src = 'image/RoIPool.png' width = '50%' height = '50%'>  

출처 : [Understanding Region of Interest — (RoI Align and RoI Warp)]  
https://towardsdatascience.com/understanding-region-of-interest-part-2-roi-align-and-roi-warp-f795196fc193

* 위 그림을 예시로 보고 3x3 RoIPool을 적용하면
* 3x3,3x3 2개인 6x3만 적용되므로
* 초록 부분은 불필요한 정보가 포함되는 부분
* 짙은 파랑과 하늘색은 정보를 잃어버리는 부분

* Mask-R-CNN의 RoIAlign : Quantization 하지 않고 RoI를 처리할 고정 size의 Feature map 생성
* RoI 영역을 pooling layer의 크기에 맞추어 등분한 후, RoIPool을 했을 때 quantization 영역 중 가까운 것들과 bilinear interpolation 계산을 통해 생성해야할 Feature map을 계산

* RoIAlign과 RoIPooling의 가장 큰 차이는 Quantization(양자화)이다.
* RoIAlign은 Quantization을 사용하지않는다.
* Fast R-CNN은 quantization을 2번 적용
  1. mapping 과정
  2. pooling 과정
* RoIAlign은 quantization을 적용하지않고 pooling layer의 size(ex 3x3)에 맞게 등분하고
* bilinear interpolation 계산을 통해 3x3에 맞는 값을 넣어줌

* 그 뒤에 Bounding box Regression 을 하는 Bbox head와 마스크를 예측하는 Mask Head 두 갈래로 나눠짐.
* Mask map의 경우 semantic segmentation과 달리 RoIAlign으로 줄어든 특성에 마스크를 예측하기 때문에 목적에 따라 정확한 마스크를 얻는 경우에는 부적합

## Interpolation

* Interpolation(보간) : 지점 사이의 값을 알려진 값으로 추정하는 것
* ex) 20때의 키와 40대의 키를 가지고 30대의 키를 추측
* Extrapolation(추론) : 1살과 현재의 키를 보고 10년 뒤의 키를 예측
* 결국 extrapolation이 안정성이 더욱 떨어진다.

### 1D Linear Interpolation


* 두 지점을 보간하는 방법은 polynomial 보간, spline 보간등이 있다.
* 그 중 linear interpolation(선형 보간)은 두 지점 사이의 값을 추정할 때
* 그 값을 두 지점과의 직선 거리에 따라 선형적으로 결정하는 방법

* 두 지점 $x_{1}, x_{2}$에서 데이터 값이 각각 $f(x_{1}), f(x_{2})$일 때
* $x_{1},x_{2}$ 사이의 임의의 지점 $x (x_{1} \leq x \leq x_{2}) $에서
* $f(x)$는 선형 보간법을 사용할 경우 다음과 같다.

$$ f(x) = \frac{d_{2}}{d_{1} + d_{2}} f(x_{1}) + \frac{d_{1}}{d_{1} + d_{2}} f(x_{2})$$

* $d_{1}$ = $x$에서$x_{1}$까지의 거리
* $d_{2}$ = $x$에서$x_{2}$까지의 거리

* 거리의 비의 합이 1이 되도록 정규화 하면
* $\alpha = d_{1}/(d_{1} + d_{2}), \beta = d_{2}/(d_{1} + d_{2}), (\alpha + \beta = 1)$
* 식이 좀더 단순화 될 수 있다.

$$f(x) = \beta f(x_{1})+\alpha f(x_{2})$$

### Bilinear Interpolation

* 쌍선형 보간법 또는 이중선형 보간법
* 1차원에서의 선형보간법을 2차원으로 확장
* 설명을 위해 직사각형의 꼭지점 값이 주어질때 내부의 임의의 점의 값을 추정하는 문제를 보자

<img src = 'image/Bilinear_interpolation.png'>

* 점 P에서 x축 방향으로 사각형 변까지의 거리를 $w_{1}, w_{2}$
* 점 P에서 y축 방향으로 거리를 $h_{1}, h_{2}$
* 4점의 데이터 값을 A, B, C, D
* $\alpha, \beta, p, q$는 비율
* $\alpha = h_{1}/(h_{1} + h_{2}), \beta = h_{2}/(h_{1} + h_{2})$
* $p = w_{1}/(w_{1} + w_{2}), q = w_{2}/(w_{1} + w_{2})$
* P에서의 데이터 값은 bilinear interpolation에 의해 다음과 같이 계산 

$$ P = q(\beta A + \alpha B) + p(\beta D + \alpha C)$$
$$ = q\beta A + q\alpha B + p\beta D + p\alpha C $$

* A, B를 보간하여 M을 얻고
* C, D를 보간하여 N을 얻고
* M, N을 보간하여 P를 얻음
* 순서를 바꾸어서 U,V를 얻은 뒤에 U,V 보간을 해도 동일한 결과

* 위의 식은 직사각형에서만 적용가능하다.
* 임의의 형태의 사각형에서는 어떻게 해야 할까?

<img src = 'image/Bilinear_interpolation_2.png'>

* 이런 경우에는 원래의 사각형을 어떤 직사각형으로 warping(워핑) 시키고
* 워핑된 사각형에서 보간을 수행한다.
* 워핑할 사각형은 임의의 사각형이 가능하지만 편의상 네 꼭지점의 좌표가
* (0,0), (0,1), (1,1), (1,0)인 단위 정사각형으로 워핑시킨다고 합시다.

<img src = 'image/Bilinear_interpolation_3.png' >

* 사각형의 네점 ABCD를 A'B'C'D'으로 변환시키는 Linear Transformation(선형변환) T를 구하고
* T를 이용해 P를 변환시킨 P'을 구하고
* 단위 정사각형에서 bilinear interpolation을 수행한다.

## FCN

* Fully Convolutional Networks for Semantic Segmentation 
* FCN(Fully Convolutional Network)
* AlexNet, VGG, GoogleNet 등을 Segmentation 목적에 맞게 변형시킨 것
* 하지만 기존 모델에서의 Fully Connected Layer는 위치정보가 사라진다.
* 또한 입력 이미지 크기가 고정된다. 
* 이를 해결하기 위해 FC layer를 Conv-layer로 대체하였다
* 이 마지막 CNN은 1x1의 kernel size와 class의 개수만큼의 채널을 가짐
* 이렇게 CNN을 거치면 class hitmap을 얻을 수 있음

* 하지만 히트맵 크기가 원본 이미지보다 작습니다.
* CNN와 Pooling을 거치면서 크기가 줄었기 때문.
* 이때 키워주는 방법이 Upsampling 입니다.
* FCN에서는 Deconvolution과 Interpolation 방식을 활용
  * Deconvolution : Convolution 연산을 거꾸로 해줌
  * Interpolation : 보간법으로 픽셀들을 추정

* Upsampling을 하면 원하는 segmentation map을 얻을 수 있습니다.
* 그것이 FCN-32s
* 하지만 추가적인 성능 향상을 위한 한가지 기법을 더해주었음
* Skip Architecture라는 방법

* Skip Architecture
* 정확하고 상세한 구분(Segmentation)을 얻기 위해
* Deep & Coarse(추상적인) layer의 의미적(Semantic) 정보와 Shallow & fine 층의
* 외관적(appearance) 정보를 결합한 Skip architecture를 정의

* 얕은 층에서는 주로 직선 및 곡선, 색상등의 낮은 수준의 특징이 활성화
* 깊은 층에서는 보다 복잡하고 포괄적인 개체 정보에 활성화
* FCNs는 Dense map에 shallow 층의 정보를 결합하는 방식으로 Segmentation 품질을 개선

## U-Net

* U-Net은 네트워크의 구조가 U자 형태를 가짐.
* FCN에서 upsampling을 통해 feature map을 키운 것을 대칭적으로 만든 것

<img src = 'image/U-Net_Architecture.png'>  

출처 : U-Net: Convolutional Networks for Biomedical paper

* U-Net은 Contracting Path와 Expanding Path로 나뉜다.
* Contracting Path : U의 좌측 부분으로 입력 이미지의 Context 포착을 목적으로 구성
* Expanding Path : U의 우측 부분으로 세밀한 Localization을 위한 구성, 높은 차원의 채널을 갖는 Up-sampling, 얕은 layer의 특징맵을 결합 - Up-sampling을 진행

* Contracting Path
* 2개의 3x3 convolution 계층, ReLu
* Downsampling을 위해서 2x2의 커널을 2 stride를 max pooling 하게 됨.
* Downsampling을 거친 후 다음 convolution 채널 크기를 2배씩 늘어나도록 설계

* Expanding Path
* 2x2 up-convolution : 채널이 절반씩 줄어들고 특성 맵의 크기는 늘어납니다.
* Contracting block과 동일하게 3x3 convolution이 2개씩 사용

* 크기와 같은 블록의 출력과 입력은 skip connection처럼 연결해주어 low-level의 feature를 활용할 수 있도록 함.
* 마지막으로 1x1 convolution으로 원하는 semantic segmentation map을 얻을 수 있습니다.
* 결과적으로, 입력으로 572x572 크기인 이미지가 들어가고 출력으로 388x388 크기에 2가지 클래스를 가진 segmentation map이 나옴.

* Downsampling과 Upsampling을 대칭으로함.
* Segmentation map resolution을 얻을 수 있음.
* tile 방식을 사용하여 어느 정도 서로 겹치는 구간을 tile로 나누어 network를 추론
* 큰 이미지에서도 높은 해상도의 segmentation map을 얻게 함.

* 세포 사이의 경계 파악
* 클래스 간 데이터 양의 불균형을 해결해주기
* 분포를 고려한 weight map을 학습때 사용
* 여기서 weight는 loss(손실 함수)에 적용되는 가중치
* 의료 영상에서 세포 내부나 배경보다는 상대적으로 면적이 작은 세포 경계를 명확하게 추론해 내는 것이 더욱 중요하기 때문에, 세포 경계의 손실에 더 많은 페널티를 부과하는 방식

## DeepLab

<img src = 'image/DeepLab.png'>  
  
출처 : Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation paper

* 위 그림은 DeepLabV3의 구조입니다.
* U-Net에서 Contracting Path와 Expanding Path 역할이 위 그림의 Encoder, Decoder 입니다.

* Encoder : 이미지에서 필요한 정보를 특성으로 추출하는 모듈
* Decoder : c